### Capstone assignment week 3

#### Q1: DataFrame conversion from wikipedia page

In [25]:
!pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [15]:
data = bs(page, 'html.parser')

In [16]:
ext = data.find('table',{'class':'wikitable sortable'})

In [19]:
print(ext.tr.text)


Postcode
Borough
Neighbourhood



In [20]:
headers="Postcode,Borough,Neighbourhood"

In [23]:
table=""
for tr in ext.find_all('tr'):
    row=""
    for tds in tr.find_all('td'):
        row=row+","+tds.text
    table=table+row[1:]

In [28]:
file=open("extraction.csv","wb")
file.write(bytes(table,encoding="ascii",errors="ignore"))

8708

In [31]:
df = pd.read_csv('extraction.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]

In [33]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [35]:
indexes_drop = df[ df['Borough'] =='Not assigned'].index
df.drop(indexes_drop , inplace=True)

In [36]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [38]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']

In [40]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [41]:
final = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)

In [44]:
final = final.reset_index()

In [47]:
final.head(12)

,index,Postalcode,Borough,Neighbourhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,Harbourfront
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,4,M7A,Downtown Toronto,Queen's Park
5,5,M9A,Queen's Park,Queen's Park
6,6,M1B,Scarborough,"Rouge, Malvern"
7,7,M3B,North York,Don Mills North
8,8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Q2: Geocoder frame

In [50]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
longlat = pd.read_csv('Toronto_long_lat_data.csv')
longlat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
longlat.columns=['Postalcode','Latitude','Longitude']

In [52]:
longlat.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
t_df = pd.merge(final,
                 longlat[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')

In [56]:
t_df

,index,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...,...
98,98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509


#### Q3: Neighborhood clusters

In [59]:
!pip install geopy
from geopy.geocoders import Nominatim as nom

     |████████████████████████████████| 112kB 24.6MB/s eta 0:00:01


In [60]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [61]:
from sklearn.cluster import KMeans


In [62]:
import folium

In [64]:
address = 'Toronto, ON'
geolocator = nom(user_agent="Toronto")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print('Coordinates: {}, {}.'.format(lat, lon))

Coordinates: 43.653963, -79.387207.


In [65]:
map_toronto = folium.Map(location=[lat, lon], zoom_start=10)

# add markers to map
for lt, lng, borough, Neighbourhood in zip(t_df['Latitude'], t_df['Longitude'], t_df['Borough'], t_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lt, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto